<a href="https://colab.research.google.com/github/m0hit-kumar/RASA/blob/main/bert_model_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd


In [ ]:
data=pd.read_json("final.json")
# data2=pd.read_json("definitions.json")


In [ ]:
data["intent"].unique()


array(['Tornado', 'volcano', 'misc', 'Earthquake', 'floods', 'War',
       'BombBlast'], dtype=object)

In [ ]:
data['intent'].value_counts()

War           2181
BombBlast     1924
floods        1433
Earthquake    1411
volcano       1369
Tornado       1339
misc           216
Name: intent, dtype: int64

In [ ]:
data=data.to_dict(orient='records')
type(data)


list

In [ ]:
import nltk
nltk.download('stopwords') # for removing useless data
nltk.download('punkt') # use unsupervised algorithm to build a model for abbreviation words, collocations, and words
nltk.download('wordnet') #used  in information systems, word-sense disambiguation, information retrieval, automatic text classification, summarization, machine translation.
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
# clean the function will apply the above mentioned preprocessing to the resumes
def clean(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('english')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    # make list of text into string
    result=" ".join(clean_text)
    return result


In [ ]:
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.8 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split

In [ ]:
 # Example data
# data = [
#     {"text": "Floods are hitting the coastal areas", "intent": "flood"},
#     {"text": "Tornado warning issued for the city", "intent": "tornado"},
#     {"text": "Earthquake devastates the region", "intent": "earthquake"},
#     # more data here
# ]

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)




In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=7)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
intent_label_map={'Tornado':0, 'misc':1, 'BombBlast':2, 'War':3, 'Earthquake':4, 'floods':5,'volcano':6}



In [ ]:
# Tokenize the data
def tokenize_data(data):
    inputs = tokenizer(data["text"], padding=True, truncation=True, return_tensors="pt")
    labels = torch.tensor([intent_label_map[data["intent"]]])  # Convert intent string to numerical label
    return inputs, labels

In [ ]:

train_data_tokenized = [tokenize_data(d) for d in train_data]
test_data_tokenized = [tokenize_data(d) for d in test_data]

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(train_data))


In [ ]:
# Training loop
epochs = 3
for epoch in range(epochs):
    for batch in train_data_tokenized:
        optimizer.zero_grad()
        outputs = model(**batch[0], labels=batch[1])
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

In [ ]:

# Evaluation
model.eval()
total = 0
correct = 0
with torch.no_grad():
    for batch in test_data_tokenized:
        outputs = model(**batch[0])
        predicted = torch.argmax(outputs.logits, dim=1)
        total += batch[1].size(0)
        correct += (predicted == batch[1]).sum().item()


In [ ]:



accuracy = correct / total
print(f"Accuracy: {accuracy}")

# Save the model
output_dir = './model_save/'
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"Model saved in {output_dir}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
